Starting this attempt small with just looking for Belgium 2023 Grand Prix

In [1]:
import time
import random
from urllib.request import urlopen
import json
import pandas as pd

def get_with_backoff(url, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = urlopen(url)
            return response
        except HTTPError as e:
            if e.code == 429:
                wait_time = (2 ** retries) + random.random()
                print(f"Rate limited. Waiting {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1
            else:
                raise
    raise Exception("Max retries exceeded")

## Meeting data

In [2]:
# specifically looking for Belgium 2023
response = urlopen('https://api.openf1.org/v1/meetings?year=2023&country_name=Belgium')
data = json.loads(response.read().decode('utf-8'))
meeting = pd.DataFrame(data)
meeting.head(10)

# meeting_key = 1216

,meeting_name,meeting_official_name,location,country_key,country_code,country_name,circuit_key,circuit_short_name,date_start,gmt_offset,meeting_key,year
0,Belgian Grand Prix,FORMULA 1 MSC CRUISES BELGIAN GRAND PRIX 2023,Spa-Francorchamps,16,BEL,Belgium,7,Spa-Francorchamps,2023-07-28T11:30:00+00:00,02:00:00,1216,2023


## Session data

In [25]:
# to find the final race data
response = urlopen('https://api.openf1.org/v1/sessions?meeting_key=1216')
data = json.loads(response.read().decode('utf-8'))
session = pd.DataFrame(data)

In [26]:
session[['session_type', 'session_name', 'session_key', 'meeting_key']].head(10)

,session_type,session_name,session_key,meeting_key
0,Practice,Practice 1,9134,1216
1,Qualifying,Qualifying,9135,1216
2,Qualifying,Sprint Shootout,9286,1216
3,Race,Sprint,9140,1216
4,Race,Race,9141,1216


So we see here the session keys corresponding to the diff sessions for this grand prix. we'll use practice and two qualifying for our training data i guess? 

## Position data

In [23]:
# search for meeting key = 1216 (Belgium 2023) 
response = urlopen('https://api.openf1.org/v1/position?meeting_key=1216')
data = json.loads(response.read().decode('utf-8'))
positions = pd.json_normalize(data)
positions_df = pd.DataFrame(data)

positions.head(20)

,position,driver_number,date,session_key,meeting_key
0,1,1,2023-07-28T11:15:04.686000+00:00,9134,1216
1,2,2,2023-07-28T11:15:04.686000+00:00,9134,1216
2,3,3,2023-07-28T11:15:04.686000+00:00,9134,1216
3,4,4,2023-07-28T11:15:04.686000+00:00,9134,1216
4,5,10,2023-07-28T11:15:04.686000+00:00,9134,1216
5,6,11,2023-07-28T11:15:04.686000+00:00,9134,1216
6,7,14,2023-07-28T11:15:04.686000+00:00,9134,1216
7,8,16,2023-07-28T11:15:04.686000+00:00,9134,1216
8,9,18,2023-07-28T11:15:04.686000+00:00,9134,1216
9,10,20,2023-07-28T11:15:04.686000+00:00,9134,1216


In [24]:
unique_session_keys = positions['session_key'].unique()
print(unique_session_keys)


[9134 9135 9286 9140 9141]


In [27]:
positions.to_csv("Belgium2023/positions_belgium2023.csv")

## MERGING

In [34]:
import pandas as pd

# Make sure 'date' is in datetime format
positions['date'] = pd.to_datetime(positions['date'])

# Get the last entry (latest date) per driver per session
latest_driver_positions = (
    positions.sort_values('date')
    .groupby(['session_key', 'driver_number'], as_index=False)
    .last()
)

# Optional: sort for easy reading
latest_driver_positions = latest_driver_positions.sort_values(
    by=['session_key', 'position']
).reset_index(drop=True)



In [35]:
latest_driver_positions

,session_key,driver_number,position,date,meeting_key
0,9134,55,1,2023-07-28 11:56:26.691000+00:00,1216
1,9134,81,2,2023-07-28 11:56:26.691000+00:00,1216
2,9134,4,3,2023-07-28 11:56:26.691000+00:00,1216
3,9134,16,4,2023-07-28 11:56:26.691000+00:00,1216
4,9134,11,5,2023-07-28 11:56:26.691000+00:00,1216
...,...,...,...,...,...
95,9286,22,16,2023-07-29 10:48:54.390000+00:00,1216
96,9286,77,17,2023-07-29 10:48:50.952000+00:00,1216
97,9286,20,18,2023-07-29 10:48:58.265000+00:00,1216
98,9286,24,19,2023-07-29 10:48:58.265000+00:00,1216


In [38]:
# Merge on session_key
merged_df = latest_driver_positions.merge(
    session[['session_key', 'session_name']],
    on='session_key',
    how='left'
)


In [39]:
merged_df

,session_key,driver_number,position,date,meeting_key,session_name
0,9134,55,1,2023-07-28 11:56:26.691000+00:00,1216,Practice 1
1,9134,81,2,2023-07-28 11:56:26.691000+00:00,1216,Practice 1
2,9134,4,3,2023-07-28 11:56:26.691000+00:00,1216,Practice 1
3,9134,16,4,2023-07-28 11:56:26.691000+00:00,1216,Practice 1
4,9134,11,5,2023-07-28 11:56:26.691000+00:00,1216,Practice 1
...,...,...,...,...,...,...
95,9286,22,16,2023-07-29 10:48:54.390000+00:00,1216,Sprint Shootout
96,9286,77,17,2023-07-29 10:48:50.952000+00:00,1216,Sprint Shootout
97,9286,20,18,2023-07-29 10:48:58.265000+00:00,1216,Sprint Shootout
98,9286,24,19,2023-07-29 10:48:58.265000+00:00,1216,Sprint Shootout


In [40]:
merged_df.to_csv("Belgium2023/final_positions.csv")